In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from numpy import array
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import r2_score, make_scorer, roc_auc_score, roc_curve
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingClassifier
import warnings
import matplotlib.pyplot as plt
from math import exp
import time
import datetime
%matplotlib  inline
warnings.simplefilter('ignore')

## Подход 1: градиентный бустинг "в лоб"

В отчете по данному этапу необходимо ответить на следующие вопросы:
1. Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?
2. Как называется столбец, содержащий целевую переменную?
3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти ниже по тексту. Какое качество при этом получилось? Напомним, что в данном задании мы используем метрику качества AUC-ROC.
4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?

Загружаем файл с данными о матчах...

In [2]:
Xcsv = pd.read_csv('out_featers.csv', index_col='match_id')

... и проверяем какие признаки содержат пропущенные данные

In [3]:
rows = Xcsv.shape[0]
for col in Xcsv.columns:
    if Xcsv[col].count() < rows:
        print(f'{col}: {Xcsv[col].count()}')
        

first_blood_time: 82494
first_blood_team: 82494
first_blood_player1: 82494
first_blood_player2: 56095
radiant_bottle_time: 81539
radiant_courier_time: 96538
radiant_flying_courier_time: 69751
radiant_first_ward_time: 95394
dire_bottle_time: 81087
dire_courier_time: 96554
dire_flying_courier_time: 71132
dire_first_ward_time: 95404


Большинство признаков с пропущенными значениями относятся к временным меткам и означают, что за первые пять минут матча соответствующее событие не произошло. В задании рекомендуется заменить такие значения на 0, но такая рекомендация в данном случае не вполне подходит. Правильнее было бы заменить эти значения, на время превосходящее или равное пяти минутам. Пропущенные значения first_blood_player1 и first_blood_player2 заменим на медианные значения по признаку. Затем проверим, что у нас больше не осталось пропущенных данных.

<b>Целевая переменная содержится в столбце 'radiant_win'.</b> Создадим из него объект ответов Y и удалим из обучающей выборки. Удалим из выборки признаки, связанные с итогами матча.

In [4]:
X = Xcsv[::]
X['first_blood_player1'].fillna(X['first_blood_player1'].median(), inplace=True)
X['first_blood_player2'].fillna(X['first_blood_player2'].median(), inplace=True)
X.fillna(300, inplace=True)
Y = X['radiant_win']
X.drop(['radiant_win',
        'duration',
        'tower_status_radiant', 
        'tower_status_dire', 
        'barracks_status_dire', 
        'barracks_status_radiant', 
        'start_time'],
       axis=1, inplace=True)
rows = X.shape[0]
for col in X.columns:
    if X[col].count() < rows:
        print(f'{col}: {Xcsv[col].count()}')
print(f'Xcsv.shape is {Xcsv.shape}')
print(f'X.shape is {X.shape}')

Xcsv.shape is (97230, 108)
X.shape is (97230, 101)


Проведём сначала решётчатый поиск на градиентном бустинге для определения оптимальных параметров. 

In [5]:
params = {'n_estimators':[10*i for i in range(1, 6)],
         'learning_rate': [i/20 for i in range(4, 11)],
         'max_depth':[i for i in range(2, 5)]}
print(*params.items(), sep='\n')

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size = 0.3,
    random_state = 241 )

cv = KFold(n_splits=5, shuffle=True, random_state=241)
scorer = make_scorer(roc_auc_score)

('n_estimators', [10, 20, 30, 40, 50])
('learning_rate', [0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5])
('max_depth', [2, 3, 4])


In [6]:
grid = GridSearchCV(GradientBoostingClassifier(random_state=241),
                    params, cv=cv, n_jobs=-1, scoring=scorer, verbose=True)
grid.fit(X_train, Y_train)

Fitting 5 folds for each of 105 candidates, totalling 525 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.6min


KeyboardInterrupt: 

Проведём кросс-валидацию на полной выборке с наилучшими значениями <i>'learning_rate'</i> и <i>'max_depth'</i>, полученными в предыдущем пункте с количеством деревьев равным 30. Измерим время кросс-валидации.

In [ ]:
clf = GradientBoostingClassifier(
    learning_rate=grid.best_params_['learning_rate'],
    max_depth=grid.best_params_['max_depth'],
    n_estimators=30,
    random_state=241, verbose=False )
%time crossval_scores = cross_val_score(clf, X, Y, cv=cv, scoring=scorer)
for i, cscore in enumerate(crossval_scores):
    print('{0}) - {1:.4f}'.format(i, cscore))


Время кросс-валидации составило  {n_estimators}

Для того чтобы ответить

In [ ]:
n_trees = [i*10 for i in range(3,101)]
#plt.figure()
roc = []
print('_'*10)
max_score = 0,0
for n in n_trees:
    clf = GradientBoostingClassifier(
        max_depth=2, n_estimators=n,random_state=241,
        learning_rate=0.3, verbose=False).fit(X_train, Y_train)
    pred = clf.predict_proba(X_test)[:, 1]
    roc_score = roc_auc_score(Y_test, pred)
    roc.append(roc_score)
    if roc_score > max_score[0]:
        max_score = roc_score, n
    if n % 100 == 0:
        print('#', end='')
plt.plot(n_trees, roc, 'b', linewidth=1)
print(f"""Наилучший результат - {max_score[0]:.3f},
количество деревьев - {max_score[1]}""")


## Подход 2: логистическая регрессия

Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?<br/>
Найдём при помощи решетчатого поиска оптимальный параметр С и проведём кросс-валидацию.

scaler = MinMaxScaler()
params = {'C' : [i/20 for i in range(10, 16)]}
Xs = scaler.fit_transform(X_train)
grid = GridSearchCV(LogisticRegression(
    random_state=241, solver='saga',
    n_jobs=-1), params, cv=cv, 
                    n_jobs=-1, scoring=scorer, verbose=True).fit(X_t, Y_train)
logreg = LogisticRegression(random_state=241, solver='saga', n_jobs=-1)
crossval_scores = cross_val_score(clf, X, Y, cv=cv, scoring=scorer)
for i, cscore in enumerate(crossval_scores):
    print('{0}) - {1:.4f}'.format(i, cscore))

Xs_test = scaler.transform(X_test)
print(s)